In [1]:
!pip install gspread

In [ ]:
import json
from datetime import datetime, timedelta
import gspread
from google.oauth2.service_account import Credentials
from google.auth.transport.requests import Request

# Function to process JSON Lines file
def process_jsonl_file(file_path, business_id, start_date, end_date):
    metrics = {
        'total_count': 0,
        'sum_stars': 0,
        'sum_useful': 0,
        'sum_funny': 0,
        'sum_cool': 0
    }

    with open(file_path, 'r') as file:
        for line in file:
            try:
                review = json.loads(line.strip())
                review_date = datetime.strptime(review['date'], '%Y-%m-%d %H:%M:%S').date()

                if start_date <= review_date <= end_date and review['business_id'] == business_id:
                    metrics['total_count'] += 1
                    metrics['sum_stars'] += review['stars']
                    metrics['sum_useful'] += review['useful']
                    metrics['sum_funny'] += review['funny']
                    metrics['sum_cool'] += review['cool']
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line[:50]}...")  # Print first 50 characters of problematic line

    if metrics['total_count'] > 0:
        return {
            'total_count': metrics['total_count'],
            'avg_stars': metrics['sum_stars'] / metrics['total_count'],
            'avg_useful': metrics['sum_useful'] / metrics['total_count'],
            'avg_funny': metrics['sum_funny'] / metrics['total_count'],
            'avg_cool': metrics['sum_cool'] / metrics['total_count']
        }
    else:
        return {
            'total_count': 0,
            'avg_stars': 0,
            'avg_useful': 0,
            'avg_funny': 0,
            'avg_cool': 0
        }

# Function to update Google Sheets
def update_google_sheets(sheet, date, metrics):
    row = [
        metrics['total_count'],
        metrics['avg_stars'],
        metrics['avg_useful'],
        metrics['avg_funny'],
        metrics['avg_cool']
    ]
    sheet.append_row(row)

# Function to get the last processed date from Google Sheets
def get_last_processed_date(sheet):
    all_values = sheet.get_all_values()
    if len(all_values) > 1:  # Check if there's data besides the header
        return datetime.strptime(all_values[-1][0], '%Y-%m-%d').date()
    return None

# Main function to process data and update Google Sheets
def process_and_update(start_date, end_date, business_id, sheet):
    file_path = "/content/drive/MyDrive/source/yelp_academic_dataset_review.json"
    try:
        metrics = process_jsonl_file(file_path, business_id, start_date, end_date)
        update_google_sheets(sheet, end_date, metrics)  # Use end_date as the date for this aggregation
        print(f"Processed data from {start_date} to {end_date}")
    except FileNotFoundError:
        print(f"No data file found at {file_path}")

# Initialize Google Sheets connection
def init_google_sheets():
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file('/content/drive/MyDrive/credentials/service_account.json', scopes=scope)
    client = gspread.authorize(creds)

    spreadsheet_name = 'Business_Reviews_Metric'

    try:
        sheet = client.open(spreadsheet_name).sheet1
        print(f"Opened existing spreadsheet: {spreadsheet_name}")
    except gspread.SpreadsheetNotFound:
        print(f"Spreadsheet '{spreadsheet_name}' not found. Creating a new one...")
        try:
            new_spreadsheet = client.create(spreadsheet_name)
            sheet = new_spreadsheet.sheet1
            print(f"Created new spreadsheet: {spreadsheet_name}")

            # Share the newly created spreadsheet with your email
            email_to_share = 'mauladiarifiqbal@gmail.com'  # Replace with your actual email
            new_spreadsheet.share(email_to_share, perm_type='user', role='writer')
            print(f"Shared spreadsheet with {email_to_share}")

        except Exception as e:
            print(f"Error creating or sharing spreadsheet: {e}")
            return None

    # Initialize headers directly
    headers = ['Total Count', 'Avg Stars', 'Avg Useful', 'Avg Funny', 'Avg Cool']

    try:
        # Fetch the first row to check for headers
        existing_headers = sheet.row_values(1)
        print("Current first row values:", existing_headers)  # Debugging line

        # Check if the first row matches the intended headers
        if existing_headers != headers:
            sheet.clear()  # Clear existing values in the sheet
            sheet.append_row(headers)  # Append new headers
            print("Initialized headers in the new spreadsheet.")
        else:
            print("Headers already exist.")

    except Exception as e:
        print(f"Error checking or adding headers: {e}")

    return sheet

# Main execution
if __name__ == "__main__":
    business_id = "7ATYjTIgM3jUlt4UM3IypQ"
    sheet = init_google_sheets()

    # Scenario 1: Initial Load
    start_date = datetime(2018, 1, 1).date()
    end_date = datetime(2018, 1, 31).date()
    process_and_update(start_date, end_date, business_id, sheet)
    print("Initial load completed")

    # Scenario 2: First Update
    start_date = datetime(2018, 2, 1).date()
    end_date = datetime(2018, 2, 2).date()
    process_and_update(start_date, end_date, business_id, sheet)
    print("First update completed")

    # Scenario 3: Delayed Update
    start_date = datetime(2018, 2, 3).date()
    end_date = datetime(2018, 2, 14).date()
    process_and_update(start_date, end_date, business_id, sheet)
    print("Delayed update completed")

    print("All scenarios processed")

Opened existing spreadsheet: Business_Reviews_Metric
Current first row values: ['Total Count', 'Avg Stars', 'Avg Useful', 'Avg Funny', 'Avg Cool']
Headers already exist.
